In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import warnings
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import ConfusionMatrixDisplay,classification_report
warnings.filterwarnings('ignore')

In [3]:
df=pd.read_csv("../input/factors-affecting-campus-placement/Placement_Data_Full_Class.csv")
df

,sl_no,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status,salary
0,1,M,67.00,Others,91.00,Others,Commerce,58.00,Sci&Tech,No,55.0,Mkt&HR,58.80,Placed,270000.0
1,2,M,79.33,Central,78.33,Others,Science,77.48,Sci&Tech,Yes,86.5,Mkt&Fin,66.28,Placed,200000.0
2,3,M,65.00,Central,68.00,Central,Arts,64.00,Comm&Mgmt,No,75.0,Mkt&Fin,57.80,Placed,250000.0
3,4,M,56.00,Central,52.00,Central,Science,52.00,Sci&Tech,No,66.0,Mkt&HR,59.43,Not Placed,NaN
4,5,M,85.80,Central,73.60,Central,Commerce,73.30,Comm&Mgmt,No,96.8,Mkt&Fin,55.50,Placed,425000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,211,M,80.60,Others,82.00,Others,Commerce,77.60,Comm&Mgmt,No,91.0,Mkt&Fin,74.49,Placed,400000.0
211,212,M,58.00,Others,60.00,Others,Science,72.00,Sci&Tech,No,74.0,Mkt&Fin,53.62,Placed,275000.0
212,213,M,67.00,Others,67.00,Others,Commerce,73.00,Comm&Mgmt,Yes,59.0,Mkt&Fin,69.72,Placed,295000.0
213,214,F,74.00,Others,66.00,Others,Commerce,58.00,Comm&Mgmt,No,70.0,Mkt&HR,60.23,Placed,204000.0


In [4]:
df['status'].value_counts()

Placed        148
Not Placed     67
Name: status, dtype: int64

**Data Preprocessing**

In [5]:
df.shape

(215, 15)

In [6]:
df.dtypes

sl_no               int64
gender             object
ssc_p             float64
ssc_b              object
hsc_p             float64
hsc_b              object
hsc_s              object
degree_p          float64
degree_t           object
workex             object
etest_p           float64
specialisation     object
mba_p             float64
status             object
salary            float64
dtype: object

In [7]:
df.isna().sum()

sl_no              0
gender             0
ssc_p              0
ssc_b              0
hsc_p              0
hsc_b              0
hsc_s              0
degree_p           0
degree_t           0
workex             0
etest_p            0
specialisation     0
mba_p              0
status             0
salary            67
dtype: int64

In [8]:
df1=df.copy()
df1.drop(['sl_no','ssc_b','hsc_b'],axis=1,inplace=True)
df1

,gender,ssc_p,hsc_p,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status,salary
0,M,67.00,91.00,Commerce,58.00,Sci&Tech,No,55.0,Mkt&HR,58.80,Placed,270000.0
1,M,79.33,78.33,Science,77.48,Sci&Tech,Yes,86.5,Mkt&Fin,66.28,Placed,200000.0
2,M,65.00,68.00,Arts,64.00,Comm&Mgmt,No,75.0,Mkt&Fin,57.80,Placed,250000.0
3,M,56.00,52.00,Science,52.00,Sci&Tech,No,66.0,Mkt&HR,59.43,Not Placed,NaN
4,M,85.80,73.60,Commerce,73.30,Comm&Mgmt,No,96.8,Mkt&Fin,55.50,Placed,425000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
210,M,80.60,82.00,Commerce,77.60,Comm&Mgmt,No,91.0,Mkt&Fin,74.49,Placed,400000.0
211,M,58.00,60.00,Science,72.00,Sci&Tech,No,74.0,Mkt&Fin,53.62,Placed,275000.0
212,M,67.00,67.00,Commerce,73.00,Comm&Mgmt,Yes,59.0,Mkt&Fin,69.72,Placed,295000.0
213,F,74.00,66.00,Commerce,58.00,Comm&Mgmt,No,70.0,Mkt&HR,60.23,Placed,204000.0


In [10]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
lst=['gender','hsc_s','degree_t','workex','specialisation','status']
for i in lst:
  df1[i]=le.fit_transform(df1[i])

In [11]:
df1

,gender,ssc_p,hsc_p,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status,salary
0,1,67.00,91.00,1,58.00,2,0,55.0,1,58.80,1,270000.0
1,1,79.33,78.33,2,77.48,2,1,86.5,0,66.28,1,200000.0
2,1,65.00,68.00,0,64.00,0,0,75.0,0,57.80,1,250000.0
3,1,56.00,52.00,2,52.00,2,0,66.0,1,59.43,0,NaN
4,1,85.80,73.60,1,73.30,0,0,96.8,0,55.50,1,425000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
210,1,80.60,82.00,1,77.60,0,0,91.0,0,74.49,1,400000.0
211,1,58.00,60.00,2,72.00,2,0,74.0,0,53.62,1,275000.0
212,1,67.00,67.00,1,73.00,0,1,59.0,0,69.72,1,295000.0
213,0,74.00,66.00,1,58.00,0,0,70.0,1,60.23,1,204000.0


In [12]:
df1.dtypes

gender              int64
ssc_p             float64
hsc_p             float64
hsc_s               int64
degree_p          float64
degree_t            int64
workex              int64
etest_p           float64
specialisation      int64
mba_p             float64
status              int64
salary            float64
dtype: object

In [13]:
#Filling the missing values in the column salary
df1['salary']=df1['salary'].fillna(df1['salary'].mode()[0])

In [14]:
X=df1.iloc[:,:10]
y=df1.iloc[:,-2]

In [15]:
#scaling the dataset using Min-Max scaler
from sklearn.preprocessing import MinMaxScaler
ms=MinMaxScaler()
X_sc=ms.fit_transform(X)
X_sc

array([[1.        , 0.53823954, 0.88962109, ..., 0.10416667, 1.        ,
        0.28448276],
       [1.        , 0.79241394, 0.68088962, ..., 0.76041667, 0.        ,
        0.56484258],
       [1.        , 0.49701093, 0.5107084 , ..., 0.52083333, 0.        ,
        0.2470015 ],
       ...,
       [1.        , 0.53823954, 0.49423394, ..., 0.1875    , 0.        ,
        0.69377811],
       [0.        , 0.68253968, 0.47775947, ..., 0.41666667, 1.        ,
        0.33808096],
       [1.        , 0.43516801, 0.34596376, ..., 0.8125    , 1.        ,
        0.33770615]])

In [16]:
#splitting the dataset to train data and test data
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_sc,y,test_size=0.2,random_state=1)

**K Neighbors classifier**

In [17]:
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train,y_train)
y_pred=knn.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.89      0.53      0.67        15
           1       0.79      0.96      0.87        28

    accuracy                           0.81        43
   macro avg       0.84      0.75      0.77        43
weighted avg       0.83      0.81      0.80        43



**Support Vector Classifier**

In [18]:
svc=SVC()
svc.fit(X_train,y_train)
y_pred1=svc.predict(X_test)
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.90      0.60      0.72        15
           1       0.82      0.96      0.89        28

    accuracy                           0.84        43
   macro avg       0.86      0.78      0.80        43
weighted avg       0.85      0.84      0.83        43



**Naive Bayes**

In [19]:
nb=GaussianNB()
nb.fit(X_train,y_train)
y_pred2=nb.predict(X_test)
print(classification_report(y_test,y_pred2))

              precision    recall  f1-score   support

           0       0.79      0.73      0.76        15
           1       0.86      0.89      0.88        28

    accuracy                           0.84        43
   macro avg       0.82      0.81      0.82        43
weighted avg       0.84      0.84      0.84        43



**Decision Tree**

In [20]:
dt=DecisionTreeClassifier(criterion='gini',random_state=1)
dt.fit(X_train,y_train)
y_pred3=dt.predict(X_test)
print(classification_report(y_test,y_pred3))

              precision    recall  f1-score   support

           0       0.69      0.60      0.64        15
           1       0.80      0.86      0.83        28

    accuracy                           0.77        43
   macro avg       0.75      0.73      0.74        43
weighted avg       0.76      0.77      0.76        43



**Random Forest**

In [21]:
rf=RandomForestClassifier(n_estimators=100,random_state=1)
rf.fit(X_train,y_train)
y_pred4=rf.predict(X_test)
print(classification_report(y_test,y_pred4))

              precision    recall  f1-score   support

           0       0.89      0.53      0.67        15
           1       0.79      0.96      0.87        28

    accuracy                           0.81        43
   macro avg       0.84      0.75      0.77        43
weighted avg       0.83      0.81      0.80        43



**AdaBoost**

In [22]:
ad=AdaBoostClassifier()
ad.fit(X_train,y_train)
y_pred5=ad.predict(X_test)
print(classification_report(y_test,y_pred5))

              precision    recall  f1-score   support

           0       1.00      0.80      0.89        15
           1       0.90      1.00      0.95        28

    accuracy                           0.93        43
   macro avg       0.95      0.90      0.92        43
weighted avg       0.94      0.93      0.93        43



**Gradient Boosting**

In [23]:
gb=GradientBoostingClassifier()
gb.fit(X_train,y_train)
y_pred6=gb.predict(X_test)
print(classification_report(y_test,y_pred6))

              precision    recall  f1-score   support

           0       0.91      0.67      0.77        15
           1       0.84      0.96      0.90        28

    accuracy                           0.86        43
   macro avg       0.88      0.82      0.83        43
weighted avg       0.87      0.86      0.85        43



**XG boosting**

In [24]:
xgb=XGBClassifier()
xgb.fit(X_train,y_train)
y_pred7=xgb.predict(X_test)
print(classification_report(y_test,y_pred7))

              precision    recall  f1-score   support

           0       0.86      0.80      0.83        15
           1       0.90      0.93      0.91        28

    accuracy                           0.88        43
   macro avg       0.88      0.86      0.87        43
weighted avg       0.88      0.88      0.88        43



Here we can see the accuracy score of AdaBoost has an accuracy of 93%

In [25]:
y.value_counts()

1    148
0     67
Name: status, dtype: int64

Here we can see that the target variable is highly imbalanced. Hence applying oversampling method which is Random **OverSampler**

In [26]:
from imblearn.over_sampling import RandomOverSampler
sm=RandomOverSampler()
X_train_sm,y_train_sm=sm.fit_resample(X_train,y_train)

In [27]:
y_train_sm.value_counts()

1    120
0    120
Name: status, dtype: int64

**K Neighbors classifier**

In [28]:
Knn_sm=KNeighborsClassifier()
Knn_sm.fit(X_train_sm,y_train_sm)
y_pred_sm=Knn_sm.predict(X_test)
print(classification_report(y_test,y_pred_sm))

              precision    recall  f1-score   support

           0       0.79      0.73      0.76        15
           1       0.86      0.89      0.88        28

    accuracy                           0.84        43
   macro avg       0.82      0.81      0.82        43
weighted avg       0.84      0.84      0.84        43



**Support Vector Machine**

In [29]:
sv_sm=SVC()
sv_sm.fit(X_train_sm,y_train_sm)
y_pred_sm1=sv_sm.predict(X_test)
print(classification_report(y_test,y_pred_sm1))

              precision    recall  f1-score   support

           0       0.67      0.80      0.73        15
           1       0.88      0.79      0.83        28

    accuracy                           0.79        43
   macro avg       0.77      0.79      0.78        43
weighted avg       0.81      0.79      0.79        43



**Naive Bayes**

In [30]:
nb_sm=GaussianNB()
nb_sm.fit(X_train_sm,y_train_sm)
y_pred_sm2=nb_sm.predict(X_test)
print(classification_report(y_test,y_pred_sm2))

              precision    recall  f1-score   support

           0       0.71      0.80      0.75        15
           1       0.88      0.82      0.85        28

    accuracy                           0.81        43
   macro avg       0.80      0.81      0.80        43
weighted avg       0.82      0.81      0.82        43



**Decision Tree Classifier**

In [31]:
dt_sm=DecisionTreeClassifier(criterion='entropy')
dt_sm.fit(X_train_sm,y_train_sm)
y_pred_sm3=dt_sm.predict(X_test)
print(classification_report(y_test,y_pred_sm3))

              precision    recall  f1-score   support

           0       0.82      0.60      0.69        15
           1       0.81      0.93      0.87        28

    accuracy                           0.81        43
   macro avg       0.82      0.76      0.78        43
weighted avg       0.81      0.81      0.81        43



**Random Forest Classifier**

In [32]:
rf_sm=RandomForestClassifier(n_estimators=100,random_state=1)
rf_sm.fit(X_train_sm,y_train_sm)
y_pred_sm4=rf_sm.predict(X_test)
print(classification_report(y_test,y_pred_sm4))

              precision    recall  f1-score   support

           0       0.80      0.80      0.80        15
           1       0.89      0.89      0.89        28

    accuracy                           0.86        43
   macro avg       0.85      0.85      0.85        43
weighted avg       0.86      0.86      0.86        43



**Adaboost**

In [34]:
ad_sm=AdaBoostClassifier()
ad_sm.fit(X_train_sm,y_train_sm)
y_pred_sm5=ad_sm.predict(X_test)
print(classification_report(y_test,y_pred_sm5))

              precision    recall  f1-score   support

           0       0.92      0.73      0.81        15
           1       0.87      0.96      0.92        28

    accuracy                           0.88        43
   macro avg       0.89      0.85      0.87        43
weighted avg       0.89      0.88      0.88        43



**Gradient Boosting**

In [36]:
gb_sm=GradientBoostingClassifier()
gb_sm.fit(X_train_sm,y_train_sm)
y_pred_sm6=gb_sm.predict(X_test)
print(classification_report(y_test,y_pred_sm6))

              precision    recall  f1-score   support

           0       0.79      0.73      0.76        15
           1       0.86      0.89      0.88        28

    accuracy                           0.84        43
   macro avg       0.82      0.81      0.82        43
weighted avg       0.84      0.84      0.84        43



**XG boost**

In [37]:
xgb_sm=XGBClassifier()
xgb_sm.fit(X_train_sm,y_train_sm)
y_pred_sm7=xgb_sm.predict(X_test)
print(classification_report(y_test,y_pred_sm7))

              precision    recall  f1-score   support

           0       0.86      0.80      0.83        15
           1       0.90      0.93      0.91        28

    accuracy                           0.88        43
   macro avg       0.88      0.86      0.87        43
weighted avg       0.88      0.88      0.88        43



After applying oversampling technique we can see that the accuracy of Random Forest has increased from 81 to 86.

But We can see that the Accuracy of Adaboost classifier remains the highest before applying oversampling which is 93%